In [1]:
import warnings
warnings.filterwarnings(action='ignore')
import pandas as pd
import numpy as np
import os
from pathlib import Path
import math
from datetime import date, datetime
from getpass import getuser
import re
import seaborn as sns
import scipy.stats as stats
from scipy.stats import pearsonr
from scipy.stats import spearmanr
import matplotlib.pyplot as plt
sns.set(style="whitegrid", context="talk")
import plotly.graph_objects as go
pd.set_option('display.max_columns', None)

In [3]:
def obtener_rutas_usuario(base_local: Path = None, base_remoto: str = None, usar_remoto: bool = False):
    """
    Devuelve las rutas de datos para el usuario actual, usando pathlib.
    Ajusta la ruta si estás ejecutando desde 'notebooks/', considerando la ruta relativa.

    Parámetros:
    - base_local: ruta base local donde se encuentran los datos, considerando la estructura cookiecutter (por defecto: '../data/')
    - base_remoto: ruta base remota tipo GCP, cuando estas en la VM
    - usar_remoto: True para rutas de cloud (base_remoto), False para rutas locales (base_local)

    Retorna:
    - dict con rutas: rawData, externalData, interimData, processedData
    """
    if base_local is None:
        try:
            base_local = Path(__file__).resolve().parent.parent / 'data'
        except NameError:
            base_local = Path.cwd()
            if 'notebooks' in base_local.parts:
                base_local = base_local.parent
            base_local = base_local / 'data'

    if base_remoto is None:
        base_remoto = 'gs://default-bucket/ScoreCI'

    if usar_remoto:
        rutas = {
            'rawData': f'{base_remoto}/raw/',
            'externalData': f'{base_remoto}/external/',
            'interimData': f'{base_remoto}/interim/',
            'processedData': f'{base_remoto}/processed/'
        }
    else:
        rutas = {
            'rawData': base_local / 'raw',
            'externalData': base_local / 'external',
            'interimData': base_local / 'interim',
            'processedData': base_local / 'processed'
        }

    return rutas

In [4]:
rutas = obtener_rutas_usuario()

In [5]:
dfHist = pd.read_parquet(os.path.join(rutas.get('rawData'), 'ent-prd-sandbox-fdo-bucket_ScoreCI_raw_BaseAnaliticaHistorico_ScoreCI_08052025_abr25v6.parquet'))

In [7]:
dfDic =  pd.read_parquet(os.path.join(rutas.get('interimData'), 'Sabana_integrada_asesor_ScoreCI_VF.parquet'))
dfene =  pd.read_parquet(os.path.join(rutas.get('interimData'), 'Sabana_integrada_asesor_ScoreCI_ene25.parquet'))
dffeb =  pd.read_parquet(os.path.join(rutas.get('interimData'), 'Sabana_integrada_asesor_ScoreCI_feb25.parquet'))
dfmar =  pd.read_parquet(os.path.join(rutas.get('interimData'), 'Sabana_integrada_asesor_ScoreCI_mar25.parquet'))
dfabr =  pd.read_parquet(os.path.join(rutas.get('interimData'), 'Sabana_integrada_asesor_ScoreCI_abr25.parquet'))

In [17]:
dfabr.head()

,ASESOR_ID,PORC_CR_T19,PORC_CR_T20,PORC_CR_T9,PORC_CR_T21,PORC_CR_T3,PORC_CR_T28,PORC_CR_T27,PORC_CR_T15,PORC_CR_T22,PORC_CR_T7,PORC_CR_T12,PORC_CR_T5,PORC_CR_T18,PORC_CR_T29,PORC_CR_T10,PORC_CR_T14,PORC_CR_T1,PORC_CR_T11,PORC_CR_T2,PORC_CR_T6,PORC_CR_T25,PORC_CR_T26,PORC_CR_T17,PORC_CR_T13,PORC_CR_T4,PORC_CR_T8,PORC_CR_T24,PORC_CR_T16,PORC_CR_T0,PORC_CR_T23,POSITION_DESC,ANT_ASESOR,C_COBRANZA_T0,C_COBRANZA_T1,C_COBRANZA_T2,HORAS_VUELO,PORCT_RENOV_T2,PORCT_RENOV_T1,PORCT_RENOV_T0,ANT_ASESOR_RANGO,ANT_ASESOR_RANGO_NUM,PCNT_FPD_T2,PCNT_FPD_T1,PCNT_FPD_T0,TOT_AMI_T2,TOT_AMI_T1,TOT_AMI_T0,TOT_MAC_T2,TOT_MAC_T1,TOT_MAC_T0,TOT_SINVC_T2,TOT_SINVC_T1,TOT_SINVC_T0,TOT_VC_T2,TOT_VC_T1,TOT_VC_T0,DESEMBOLSOS_SIN_EXP_T2,DESEMBOLSOS_CON_EXP_T2,DESEMBOLSOS_SIN_EXP_T1,DESEMBOLSOS_CON_EXP_T1,DESEMBOLSOS_SIN_EXP_T0,DESEMBOLSOS_CON_EXP_T0,AVG_CICLOS_EJE_T2,AVG_CICLOS_EJE_T1,AVG_CICLOS_EJE_T0,AVG_CICLOS_CI_T2,AVG_CICLOS_CI_T1,AVG_CICLOS_CI_T0,CLIENTES_EXPERIENCIA_GRUPAL_T2,CLIENTES_EXPERIENCIA_GRUPAL_T1,CLIENTES_EXPERIENCIA_GRUPAL_T0,NUEVOS_PRODUCTO_T2,NUEVOS_PRODUCTO_T1,NUEVOS_PRODUCTO_T0,NUEVOS_PUROS_T2,NUEVOS_PUROS_T1,NUEVOS_PUROS_T0,SEGURO_FAMILIAR_T2,SEGURO_FAMILIAR_T1,SEGURO_FAMILIAR_T0,SEGURO_VOLUNTARIO_T2,SEGURO_VOLUNTARIO_T1,SEGURO_VOLUNTARIO_T0,CTES0_SIN_EXP_T2,CTES0_CON_EXP_T2,CTES1A7_SIN_EXP_T2,CTES1A7_CON_EXP_T2,CTES8A60_SIN_EXP_T2,CTES8A60_CON_EXP_T2,CTES60M_SIN_EXP_T2,CTES60M_CON_EXP_T2,CTES0_SIN_EXP_T1,CTES0_CON_EXP_T1,CTES1A7_SIN_EXP_T1,CTES1A7_CON_EXP_T1,CTES8A60_SIN_EXP_T1,CTES8A60_CON_EXP_T1,CTES60M_SIN_EXP_T1,CTES60M_CON_EXP_T1,CTES0_SIN_EXP_T0,CTES0_CON_EXP_T0,CTES1A7_SIN_EXP_T0,CTES1A7_CON_EXP_T0,CTES8A60_SIN_EXP_T0,CTES8A60_CON_EXP_T0,CTES60M_SIN_EXP_T0,CTES60M_CON_EXP_T0,CR0_SIN_EXP_T2,CR0_CON_EXP_T2,CR1A7_SIN_EXP_T2,CR1A7_CON_EXP_T2,CR8A60_SIN_EXP_T2,CR8A60_CON_EXP_T2,CR60M_SIN_EXP_T2,CR60M_CON_EXP_T2,CR0_SIN_EXP_T1,CR0_CON_EXP_T1,CR1A7_SIN_EXP_T1,CR1A7_CON_EXP_T1,CR8A60_SIN_EXP_T1,CR8A60_CON_EXP_T1,CR60M_SIN_EXP_T1,CR60M_CON_EXP_T1,CR0_SIN_EXP_T0,CR0_CON_EXP_T0,CR1A7_SIN_EXP_T0,CR1A7_CON_EXP_T0,CR8A60_SIN_EXP_T0,CR8A60_CON_EXP_T0,CR60M_SIN_EXP_T0,CR60M_CON_EXP_T0,AVG_ATRASO_ACT_EXP_T2,AVG_ATRASO_ACT_EXP_T1,AVG_ATRASO_ACT_EXP_T0,AVG_ATRASO_ACT_SIN_EXP_T2,AVG_ATRASO_ACT_SIN_EXP_T1,AVG_ATRASO_ACT_SIN_EXP_T0,TIME_BUSINESS_INDEFINIDO,TIME_BUSINESS_DEFAULT,TIME_BUSINESS_2,TIME_BUSINESS_3,TIME_BUSINESS_4,TIME_BUSINESS_5,EDAD_MASCULINO,EDAD_FEMENINO,PORC_FEMENINO,SECTOR_8,SECTOR_22,SECTOR_28,SECTOR_30,MONTO_PRESTADO_PROMEDIO_SE_T2,MONTO_PRESTADO_PROMEDIO_CE_T2,MONTO_PRESTADO_PROMEDIO_SE_T1,MONTO_PRESTADO_PROMEDIO_CE_T1,MONTO_PRESTADO_PROMEDIO_SE_T0,MONTO_PRESTADO_PROMEDIO_CE_T0,PCNT_DESEMBOLSO_TDD,DETERIORO8A60_T2,DETERIORO8A60_T1,DETERIORO8A60_T0,SE_MANTIENE_0_PCT_T0,SE_MANTIENE_ATRASO_PCT_T0,MEJORA_PCT_T0,EMPEORA_T0,SE_MANTIENE_0_PCT_T1,SE_MANTIENE_ATRASO_PCT_T1,MEJORA_PCT_T1,EMPEORA_T1,SE_MANTIENE_0_PCT_T2,SE_MANTIENE_ATRASO_PCT_T2,MEJORA_PCT_T2,EMPEORA_T2,SE_MANTIENE_0_PCT_CATS_T0,SE_MANTIENE_ATRASO_PCT_CATS_T0,MEJORA_PCT_CATS_T0,EMPEORA_CATS_T0,SE_MANTIENE_0_PCT_CATS_T1,SE_MANTIENE_ATRASO_PCT_CATS_T1,MEJORA_PCT_CATS_T1,EMPEORA_CATS_T1,SE_MANTIENE_0_PCT_CATS_T2,SE_MANTIENE_ATRASO_PCT_CATS_T2,MEJORA_PCT_CATS_T2,EMPEORA_CATS_T2
0,1949827,0.000000,0.000000,0.048167,0.000000,0.020897,0.000000,0.000000,0.095078,0.000000,0.043097,0.019886,0.033791,0.000000,0.000000,0.037748,0.057905,0.033106,0.048462,0.042283,0.059170,0.00000,0.00000,0.000000,0.037075,0.018219,0.035683,0.000000,0.093738,0.046429,0.000000,ASESOR CI,253,6.0,3.0,8.0,290,0.125000,0.0,0.250000,g.>=37,6,0.152000,0.148760,0.155963,2,2,2,125,126,130,120.0,121.0,123.0,5.0,5.0,7.0,14,0,12,1,13,3,4.792000,4.706349,5.307692,2.200000,2.269841,2.346154,29,27,30,7,3,6,6,3,3,91,79,68,93,81,70,104,15,0,0,4,0,2,0,108,14,0,0,3,1,0,0,106,14,0,0,8,2,0,0,4001089.94,1199530.11,0.00,0.00,184731.92,0.00,44873.75,0.0,4648576.31,1325116.39,0.00,0.00,184632.25,19902.36,0.0,0.0,5093836.26,1500735.00,0.00,0.00,289462.08,31627.25,0.00,0.0,2.200000,0.909910,2.201754,0.000000,1.600000,5.187500,0

In [19]:
dfabr[['ASESOR_ID','PORC_CR_T0']].to_clipboard()

In [20]:
dfHist.head()

,EMPLOYEE_ID,EMPLOYEE_PAYROLL_NUM,CARTERA_ASESOR,CR1_7,CR8_8_60,CRMAS_60,SINATRASO,CORTE
0,45513062,93252,6389333.66,0.0,194544.23,0.0,6194789.43,2024-12-31
1,81665187,119275,7000777.81,0.0,144604.19,0.0,6856173.62,2024-12-31
2,67270285,103154,5691509.78,0.0,120305.99,0.0,5571203.79,2024-12-31
3,27651448,74844,5526615.41,0.0,245749.16,0.0,5280866.25,2024-12-31
4,22184455,56812,11900248.15,0.0,238620.99,0.0,11661627.16,2024-12-31


In [21]:
dfHist[['EMPLOYEE_ID','EMPLOYEE_PAYROLL_NUM']].to_clipboard()